In [1]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

In [2]:
vector_dimension = 100

In [3]:
vector_a = np.random.randint(vector_dimension, size=vector_dimension) 
vector_b = np.random.randint(vector_dimension, size=vector_dimension)

In [4]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [5]:
mf = cl.mem_flags 
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_a) 
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_b) 

In [6]:
program = cl.Program(context, """ 
__kernel void vectorSum(__global const int *a_g, __global const int *b_g, __global int *res_g) { 
  int gid = get_global_id(0); 
  res_g[gid] = a_g[gid] + b_g[gid]; 
} 
""").build()

In [7]:
res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)

In [8]:
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

In [9]:
res_np = np.empty_like(vector_a)

In [10]:
cl.enqueue_copy(queue, res_np, res_g)

In [11]:
print ("PyOPENCL SUM OF TWO VECTORS")
print ("Platform Selected = %s" %platform.name )
print ("Device Selected = %s" %device.name)
print ("VECTOR LENGTH = %s" %vector_dimension)
print ("INPUT VECTOR A")
print (vector_a)
print ("INPUT VECTOR B")
print (vector_b)
print ("OUTPUT VECTOR RESULT A + B ")
print (res_np)

PyOPENCL SUM OF TWO VECTORS
Platform Selected = Apple
Device Selected = Apple M1
VECTOR LENGTH = 100
INPUT VECTOR A
[96  0 64 45 77 87 34 30  9 75 47 14 28 81  3 63 57  9 31 35 92  1 16 96
 13 20 54 69 35 22 96 76 49 66 75 95 33 36 71 66 84 70 57 79 56 88 49  1
 97 34  7  7 98 49  5 26  7 10 24 10 31 29 50  8 98 22 75 79 76 74 22 37
 14 88 42  4 40 17 35 84 91 51 91 35 95 41 69 87 48 10 73 32 86 93 80  5
 60 10 66 80]
INPUT VECTOR B
[76 36 75 27 14 54 36 78 68 69 73 71 97 80  1 29  4 51 45 61 37 57  5  6
 11 78 95  4 56 81 70 17  4 57 92 76 83 73  6 15 64 12  3  6 17 54 67 11
 23 54 74 85 28 44 56 95 49 89 28 47  3 31 24 25 96 45 82 87 38 55 14 16
  4 99 19 44 83 47 77 75 30 41 98 43 12 81  8 76 98 25 45 21 23  9 13 44
 71 90 33 51]
OUTPUT VECTOR RESULT A + B 
[172  36 139  72  91 141  70 108  77 144 120  85 125 161   4  92  61  60
  76  96 129  58  21 102  24  98 149  73  91 103 166  93  53 123 167 171
 116 109  77  81 148  82  60  85  73 142 116  12 120  88   0   0   0   0
   0   0  

In [12]:
assert(la.norm(res_np - (vector_a + vector_b))) < 1e-5

AssertionError: 